<a href="https://colab.research.google.com/github/Gaukhar-ai/for_my_Thinkful_work/blob/master/hamming_student_stress_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Day 50 Lecture 1 Assignment

In this assignment, we will calculate affinity propagation clustering using responses to a survey about student life at a university.

In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.cluster import AffinityPropagation
from scipy.spatial.distance import pdist, squareform

We will load a student life survey dataset. This dataset consists of 35 binary features, each corresponding to a yes/no question that characterizes the student taking the survey.

This dataset contains a large number of features, each corresponding to a survey question. The feature name summarizes the survey question, so we will not list them all out here.

In [3]:
# answer goes here

df = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/student_life_survey.csv')
df



,Q2-Participated in Societies and Interest Groups,Q2-Participated in Clubs,"Q2-Participated in Halls, JCRCs and/or Residential College CSCs",Q2-Participated in University organised events,Q3-Interested in Arts & Culture,Q3-Interested in Science & Technology,Q3-Interested in Research and independent study,Q3-Interested in Sports,"Q3-Interested in Other competitions (eg case, debates)",Q3-Interested in Entrepreneurship,Q3-Interested in Volunteering,Q3-Interested in Others,Q4-Passionate about Animal welfare,Q4-Passionate about Arts/Culture/Heritage,Q4-Passionate about Children/Youth,Q4-Passionate about Community building,"Q4-Passionate about Diversity & Inclusion (e.g. special needs, migrant worker, interfaith and intercultural understanding)",Q4-Passionate about Environmental sustainability,Q4-Passionate about Families,Q4-Passionate about Health/Well-being (e.g mental health),Q4-Passionate about Seniors,Q4-Passionate about Poverty reduction,Q4-Passionate about Education,Q4-Passionate about None of the above,Q4-Passionate about Others,Q5-Stressed about Adjustment issues,Q5-Stressed about Academic issues,Q5-Stressed about Financial issues,Q5-Stressed about Family issues,Q5-Stressed about Friendships,Q5-Stressed about Romantic relationships,Q5-Stressed about Health related issues,Q5-Stressed about Career related issues,"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",Q5-Stressed about Others,response_id
0,0,1,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,2
2,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,3
3,1,1,1,1,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,1,0,1,1,1,1,0,1,0,4
4,1,0,1,1,0,1,1,0,0,1,1,0,0,0,1,1,0,1,0,1,0,0,1,0,0,0,1,1,0,1,0,0,0,1,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2953,1,0,0,0,1,1,1,1,0,0,1,0,1,1,0,0,0,1,1,1,1,1,1,0,0,0,1,1,0,0,1,0,0,0,0,2954
2954,1,0,0,0,0,1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,2955
2955,1,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2956
2956,0,1,0,1,1,1,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0,0,2957


For our analysis, we will focus on a specific subset of the survey that is focused on stress. These questions all begin with the string 'Q5'. Filter the columns that meet this criteria (should be 10 in total).

In addition, we are only going to perform clustering on a random subset of this data, as affinity propagation is a fairly slow algorithm and requires infeasibly long times to converge for even medium-sized datasets. Select a random sample of 500 rows from the dataset.

In [8]:
# answer goes here
q5 = df.filter(like='Q5')
q5_sample = q5.sample(500, random_state=55)
q5.head()


,Q5-Stressed about Adjustment issues,Q5-Stressed about Academic issues,Q5-Stressed about Financial issues,Q5-Stressed about Family issues,Q5-Stressed about Friendships,Q5-Stressed about Romantic relationships,Q5-Stressed about Health related issues,Q5-Stressed about Career related issues,"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",Q5-Stressed about Others
0,0,1,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,1,0,0
3,1,1,0,1,1,1,1,0,1,0
4,0,1,1,0,1,0,0,0,1,0


The sklearn implementation of affinity propagation only supports euclidean and precomputed distances, so we will need to precompute a dissimilarity matrix. Furthermore, it expects negative values; the default affinity is negative euclidean distance. 

Compute the full dissimilarity matrix between all pairs of students using the negative matching/hamming distance and store it in a dataframe. 

Note: Be sure to convert the values to negative to match what the algorithm expects.

In [11]:
# answer goes here
distance = squareform(pdist(q5_sample, metric = 'hamming'))
affinity_matrix = distance



Using the dissimilarity matrix and the specified preference value, run affinity propagation on the survey results using the default value for preference, which is the median dissimilarity, and a damping parameter of 0.8. How many exemplars did it identify? If there are too many exemplars, what changes would we want to make?

In [29]:
# answer goes here
cluster = AffinityPropagation(
    damping = 0.8,
    affinity = 'precomputed', 
    max_iter = 500,
    preference = -7
)
cluster.fit(affinity_matrix)



AffinityPropagation(affinity='precomputed', convergence_iter=15, copy=True,
                    damping=0.8, max_iter=500, preference=-7, verbose=False)

In [30]:
q5_sample['label'] = cluster.labels_

In [31]:
q5_sample['label'].value_counts()

3    175
1    122
0    113
2     90
Name: label, dtype: int64

In [26]:
affinity_matrix

array([[0. , 0.3, 0.2, ..., 0.2, 0. , 0.3],
       [0.3, 0. , 0.3, ..., 0.1, 0.3, 0.2],
       [0.2, 0.3, 0. , ..., 0.2, 0.2, 0.1],
       ...,
       [0.2, 0.1, 0.2, ..., 0. , 0.2, 0.1],
       [0. , 0.3, 0.2, ..., 0.2, 0. , 0.3],
       [0.3, 0.2, 0.1, ..., 0.1, 0.3, 0. ]])

In [27]:
cluster.get_params

<bound method BaseEstimator.get_params of AffinityPropagation(affinity='precomputed', convergence_iter=15, copy=True,
                    damping=0.8, max_iter=500, preference=50, verbose=False)>

In [25]:
q5_sample['label'] = cluster.labels_

Try adjusting the value of the preference based on the result you saw in the previous step until you have a reasonable number of exemplars. Print out the data for each of these exemplars, as well as the number of surveys assigned to each exemplar. How do these clusters compare to what we saw previously with k-medoids?

Tip: large preferences can lead to numerical instability and issues with convergence. The "damping" parameter can help control this by downscaling the impact of incoming messages; check the documentation for AffinityPropagation for more details().

In [33]:
# answer goes here
cluster_avg = q5_sample.groupby('label').mean().T
cluster_avg.style.background_gradient(axis=1)




label,0,1,2,3
Q5-Stressed about Adjustment issues,0.159292,0.729508,0.644444,0.057143
Q5-Stressed about Academic issues,0.982301,0.991803,0.966667,0.817143
Q5-Stressed about Financial issues,0.814159,0.213115,0.588889,0.131429
Q5-Stressed about Family issues,0.504425,0.139344,0.111111,0.028571
Q5-Stressed about Friendships,0.221239,0.229508,0.700000,0.108571
Q5-Stressed about Romantic relationships,0.132743,0.426230,0.122222,0.017143
Q5-Stressed about Health related issues,0.336283,0.081967,0.311111,0.034286
Q5-Stressed about Career related issues,0.601770,0.565574,0.177778,0.440000
"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",0.274336,0.368852,0.322222,0.234286
Q5-Stressed about Others,0.017699,0.008197,0.000000,0.022857
